# Add some code on ops.py
 on "coremltools/converters/mil/frontend/torch/ops.py" file, add some code.

In [ ]:
@register_torch_op
def dim(context, node):
    inputs = _get_inputs(context, node)
    shape_node = mb.shape(x=inputs[0], name = node.name)

    context.add(shape_node)

# Case 1. Simple Model Test

## Create Torch Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleTest(nn.Module):
    def __init__(self):
        super(SimpleTest, self).__init__()
        self.fc1 = nn.Linear(400, 10)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

simple_model = SimpleTest()

print(simple_model)


## Script to model

In [ ]:
import coremltools

scripted_model = torch.jit.script(simple_model)
mlmodel = coremltools.converters.convert(scripted_model, inputs=[coremltools.TensorType(shape=(400,))])


## Trace to model

In [ ]:
import coremltools

simple_model.eval()

example_input = torch.rand(400)
output = simple_model(example_input)

trace = torch.jit.trace(simple_model, example_input)

mlmodel = coremltools.convert(trace, source= 'pytorch', inputs=[coremltools.TensorType(name="input", shape=example_input.shape)])


## Save ML Model

In [ ]:
mlmodel.save('simple_model_1.mlmodel')


# Case 2. Simple Model Test 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simple layer module we'll reuse in our network.
class Layer(nn.Module):
    def __init__(self, dims):
        super(Layer, self).__init__()
        self.conv1 = nn.Conv2d(*dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2,2))
        return x

In [ ]:
# A simple network consisting of several base layers.
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = Layer((3, 6, 1))
        self.layer2 = Layer((6, 16, 1))
        self.fc1 = nn.Linear(16 * 5 * 5, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(1, -1)
        x = self.fc1(x)
        return x

In [ ]:
model = SimpleNet()
print(model)

## convert

In [ ]:
import coremltools
model.eval()

traceable_model = model
example_input = torch.rand(1,3,20,20)
output = model(example_input)

print(output.shape)

trace = torch.jit.trace(traceable_model, example_input)

# print(trace)

mlmodel = coremltools.convert(trace, source= 'pytorch', inputs=[coremltools.TensorType(name="xyz", shape=example_input.shape)])

In [ ]:
mlmodel.save('test3.mlmodel')

# Case 3. MNIST Model

## import needed modules

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F

## make model

In [10]:
# LeNet
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    # 입력 이미지 채널 1개, 출력 채널 6개, 3x3의 정사각 컨볼루션 행렬
    # 컨볼루션 커널 정의
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)
    # 아핀(affine) 연산 : y = Wx + b
    self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6은 이미지 차원
    # self.fc1 = nn.Linear(576, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    # self.showshape = True

  def forward(self, x):
    # (2, 2) 크기 윈도우에 대해 맥스 풀링(max pooling)
    #input 28x28 >> after conv 6@26x26 >> max pooling 6@13x13
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # 크기가 제곱수라면 하나의 숫자만을 특정
    #input 6@13x13 >> after conv 16@11x11 >> max pooling 16@5x5
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    # x = x.view(-1, self.num_flat_features(x))
    x = x.view(-1, 576)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
  def num_flat_features(self, x):
    #print(x.size())
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    #print(num_features)
    return num_features

net = Net()#.to(device)
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Save Model

In [ ]:
torch.save(net, './nntest.pt')

## Load Model

In [11]:
net = torch.load('./nntest.pt')

## Load MNIST datas

In [4]:
import torchvision.transforms as transforms

# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5,), (1.0,))])

from torchvision.datasets import MNIST

download_root = './MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)


from torch.utils.data import DataLoader

# option 값 정의
batch_size = 64

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

## Train

In [ ]:
import numpy as np
import torch.optim as optim

net.train()

train_loss = []
train_accu = []

i = 0

for epoch in range(15):
  for data, target in train_loader:

    #data = data.to(device)
    #target = target.to(device)
    optimizer = optim.SGD(net.parameters(), lr = 0.02)
    optimizer.zero_grad()
    output = net(data)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(output, target)
    loss.backward() # calc gradients
    train_loss.append(loss.data)
    optimizer.step() # update gradients
    prediction = output.data.max(1)[1]
    accuracy = prediction.eq(target.data).sum() / batch_size * 100
    train_accu.append(accuracy)
    if i % 1000 == 0:
      print(f'Train Step: {i}\tLoss: {loss.data}\tAccuracy: {accuracy}')
    i += 1

## Check loaded model

In [6]:
net.eval()
count = 0
accuracy = 0.0
for data, target in test_loader:#valid_loader:
  #print(data.shape)
  #print(target.shape)
  #data, target = data.to(device), target.to(device)
  output = net(data)
  criterion = nn.CrossEntropyLoss()
  loss = criterion(output, target)
  prediction = output.data.max(1)[1]
  accuracy += prediction.eq(target.data).sum()
  count += prediction.shape[0]
accuracy = accuracy / count * 100
print(f'Loss: {loss.data}\tAccuracy: {accuracy}')

Loss: 0.0006188311381265521	Accuracy: 98.3499984741211


## Convert Model

In [12]:
# print(data.shape)
input_tensor = mnist_transform(data[0][0].numpy())
print(input_tensor.shape)
input_batch = input_tensor.unsqueeze(0)
print(input_batch.shape)

torch.Size([1, 28, 28])
torch.Size([1, 1, 28, 28])


In [13]:
import coremltools
net.eval()
traceable_model = net
# example_input = torch.rand(1,1,28,28)
trace = torch.jit.trace(traceable_model, input_batch)

# print(trace)

mlmodel = coremltools.convert(trace, source= 'pytorch', inputs=[coremltools.TensorType(name="image_input", shape=input_batch.shape)])

# mlmodel = coremltools.convert("nntest.pt", source= 'pytorch', inputs=[coremltools.TensorType(name="input", shape=(1,1,28,28))])

Translating MIL ==> MLModel Ops: 100%|██████████| 41/41 [00:00<00:00, 2144.81 ops/s]11 constant
12 constant
13 constant
14 constant
15 listconstruct
16 listconstruct
17 listconstruct
18 listconstruct
input.2 _convolution
input.3 relu
21 constant
22 constant
23 listconstruct
24 listconstruct
25 constant
26 constant
27 listconstruct
28 constant
29 constant
30 listconstruct
31 constant
input.4 max_pool2d
33 constant
34 constant
35 constant
36 constant
37 listconstruct
38 listconstruct
39 listconstruct
40 listconstruct
input.5 _convolution
input.6 relu
43 constant
44 constant
45 listconstruct
46 listconstruct
47 constant
48 constant
49 listconstruct
50 constant
51 constant
52 listconstruct
53 constant
x max_pool2d
55 constant
56 constant
57 listconstruct
input.7 view
59 constant
60 t
input.8 addmm
input.9 relu
63 constant
64 t
input.10 addmm
input relu
67 constant
68 t
69 addmm



## from model script

In [9]:
#model = ControlFlowNet(num_channels=3)
scripted_model = torch.jit.script(net)

import coremltools
mlmodel = coremltools.converters.convert(
  scripted_model,
  inputs=[coremltools.TensorType(shape=(1, 1, 28, 28))],
)

Converting Frontend ==> MIL Ops:   0%|          | 0/1 [00:00<?, ? ops/s]

Converting Frontend ==> MIL Ops:  32%|███▏      | 17/53 [00:00<00:00, 1464.25 ops/s]
11 constant
12 constant
13 constant
14 constant
15 constant
16 constant
17 constant
18 constant
19 listconstruct
20 listconstruct
21 listconstruct
22 conv2d
result.3 if
result.4 relu_
result.5 relu
26 listconstruct
27 listconstruct
28 listconstruct
29 constant
30 __is__


RuntimeError: PyTorch convert function for op '__is__' not implemented.

## save ml model

In [43]:
mlmodel.save("mnist_traced.mlmodel")

In [47]:
import json

# load the model
mlmodel = coremltools.models.MLModel("mnist_traced.mlmodel")

labels_json = {"labels": [0,1,2,3,4,5,6,7,8,9]}

mlmodel.user_defined_metadata["com.apple.coreml.model.preview.type"] = "MNIST"
mlmodel.user_defined_metadata['com.apple.coreml.model.preview.params'] = json.dumps(labels_json)

mlmodel.save("mnist_traced.mlmodel")